PART A:
1. Write programs to calculate info-gain, gain-ratio, gini-split, miss-classification error, and chi-square value for all features (all 56 datasets).
2. Rank the features using above metrics.



In [1]:
import pandas as pd
import numpy as np

## Gini Split Values

In [ ]:
def nos(c,in1):
    in2=np.where(c[in1[0]]==0)
    in3=np.where(c[in1[0]]>0)
    return len(in2[0]),len(in3[0])

def giniscal(f,c):
    mf=np.median(f)
    in1=np.where(f<=mf)
    in2=np.where(f>mf)
    n00,n01=nos(c,in1)
    n10,n11=nos(c,in2)
    if (n00==0 or n01==0):
        p00=0.5
        p01=0.5
    else:
        p00=n00/(n00+n01)
        p01=n01/(n00+n01)
    if(n10==0 or n11==0):
        p10=0.5
        p11=0.5
    else:
        p10=n10/(n10+n11)
        p11=n11/(n10+n11)
    gl=1-p00*p00-p01*p01;
    gr=1-p10*p10-p11*p11;
    fgs=len(in1[0])/(len(in1[0])+len(in2[0]))*gl+len(in2[0])/(len(in1[0])+len(in2[0]))*gr
    return fgs

gsv=np.zeros((56,20))
# fileloc='C:/users/lov/Desktop/tsdata/'
for i in range(1,57):
    print(i)
    fname=str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    for j in range(0,21):
        gsv[i-1,j]=giniscal(data[:,j],data[:,-1])
        gsv.sort()

In [24]:
df = pd.DataFrame(gsv)
df.to_csv("ginisplit.csv")

## Info GAIn

In [40]:
def info_gain(groups, classes, dataset):
    n_instances = float(sum([len(group) for group in groups]))
    infogain = 0.0
    for class_val in classes:
            p = [row[-1] for row in dataset].count(class_val) / n_instances
            if p == 0.0:
              continue
            infogain += - p * np.log2(p)
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            if p == 0.0:
              continue
            score += - p * np.log2(p)
        infogain -= (score) * (size / n_instances)
    return infogain

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split_info_gain(dataset):
    class_values = list(set(row[-1] for row in dataset))
    info_all=[]
    for index in range(len(dataset[0])-1):
        value = np.mean([row[index] for row in dataset])
        groups = test_split(index, value, dataset)
        info = info_gain(groups, class_values,dataset)
        info_all.append(info)
    # info_all.sort(key= lambda x: x[1],reverse =True)
    return info_all

lis = []
for i in range(1,57):
    fname = str(i)+".csv"
    X = np.genfromtxt(fname, delimiter=',')
    lis.append(get_split_info_gain(X))
    # X = X[:,0:-1]
    # np.savetxt(str(i)+"info.csv",get_split_info_gain(X), delimiter=',', fmt='%f')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('acc.csv', header = None)
print(df.head(3))
df.columns = ['Model1', 'Model2', 'Model3', 'Model4', 'Model5']
print(df.head(3))

sns.boxplot(data=df,palette="colorblind")
plt.savefig('accuracy.png')

## Miss classification

In [ ]:
def mc_mce(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    mce = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        prob = []
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            prob.append(p)
        mce += (1.0 - np.amax(prob)) * (size / n_instances)
    return mce

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split_mc_mce(dataset):
    class_values = list(set(row[-1] for row in dataset))
    mce_all=[]
    for index in range(len(dataset[0])-1):
        value=np.mean([row[index] for row in dataset])
        groups = test_split(index, value, dataset)
        mce = mc_mce(groups, class_values)
        mce_all.append([index,mce])
    # mce_all.sort(key= lambda x: x[1])
    return mce_all

for i in range(1,57):
    fname = str(i)+".csv"
    X = np.genfromtxt(fname, delimiter=',')
    # X = X[:,0:-1]
    np.savetxt(str(i)+"mce.csv",get_split_mc_mce(X), delimiter=',', fmt='%f')
    print(i)

## Chi Square

In [34]:
def chi_square(groups, classes,dataset):
    count_group=[len(group) for group in groups]
    n_instances = float(sum(count_group))
    count_class=[]
    for class_val in classes:
            count_class.append([row[-1] for row in dataset].count(class_val))
    expected_matrix=[c*g/n_instances for g in count_group for c in count_class]
    chi = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        prob = []
        for class_val in classes:
            n = [row[-1] for row in group].count(class_val) 
            try:
                ind=(groups.index(group)*2 + classes.index(class_val))
            except ValueError:
                continue
            if expected_matrix[ind] == 0:
               expected_matrix[ind] = 1
            ind=(groups.index(group)*2 + classes.index(class_val))
            chi+= (n-expected_matrix[ind])**2/expected_matrix[ind]
    return chi

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split_chi_square(dataset):
    class_values = list(set(row[-1] for row in dataset))
    chi_all=[]
    for index in range(len(dataset[0])-1):
        value=np.mean([row[index] for row in dataset])
        groups = test_split(index, value, dataset)
        chi = chi_square(groups, class_values,dataset)
        chi_all.append([index,chi])
    chi_all.sort(key= lambda x: x[1],reverse=True)
    return [c[0] for c in chi_all][:5]

lis = []
for i in range(1,57):
    fname = str(i)+".csv"
    X = np.genfromtxt(fname, delimiter=',')
    # np.savetxt(str(i)+"chi.csv",get_split_chi_square(X), delimiter=',', fmt='%f')
    lis.append(get_split_chi_square(X))
print(lis)

[[11, 13, 1, 14, 15], [11, 13, 14, 1, 7], [11, 13, 14, 1, 15], [11, 13, 14, 1, 15], [13, 11, 14, 1, 15], [11, 9, 14, 17, 19], [11, 14, 13, 9, 19], [11, 13, 19, 14, 12], [11, 1, 13, 19, 14], [11, 1, 14, 19, 13], [11, 1, 13, 19, 14], [19, 13, 11, 14, 1], [11, 14, 3, 19, 6], [11, 14, 4, 8, 0], [11, 14, 1, 13, 12], [11, 14, 1, 13, 12], [13, 11, 14, 15, 9], [13, 11, 14, 15, 17], [13, 11, 14, 15, 17], [13, 11, 14, 15, 17], [11, 13, 14, 17, 19], [13, 19, 9, 3, 1], [14, 12, 17, 19, 4], [14, 17, 12, 18, 19], [14, 11, 1, 13, 12], [1, 11, 13, 14, 7], [1, 11, 13, 14, 15], [1, 11, 13, 15, 7], [2, 9, 14, 0, 6], [14, 13, 17, 9, 7], [14, 1, 13, 11, 9], [13, 14, 1, 15, 9], [13, 14, 9, 7, 1], [13, 14, 9, 7, 1], [13, 14, 1, 7, 0], [11, 14, 7, 9, 0], [11, 14, 19, 4, 18], [13, 14, 11, 7, 9], [2, 9, 14, 11, 0], [11, 7, 6, 3, 4], [11, 7, 4, 1, 13], [11, 14, 1, 7, 13], [11, 14, 1, 7, 13], [2, 11, 13, 1, 14], [2, 11, 1, 13, 15], [14, 9, 8, 12, 6], [7, 11, 14, 1, 13], [11, 17, 9, 1, 13], [1, 13, 14, 9, 11], [6,

In [35]:
lis = np.array(lis)

In [36]:
np.savetxt("chiranked.csv",lis, delimiter=',', fmt='%f') 

## Gain Ratio

In [ ]:
def nos(c,in1):
    in2=np.where(c[in1[0]]==0)
    in3=np.where(c[in1[0]]>0)
    return len(in2[0]),len(in3[0])

def val(a,b,x):
    rval = (a+b)/x
    ans = -rval*(np.math.log(rval,2))
    return ans


def splitinfo(f,c):
    mf=np.median(f)
    in1=np.where(f<=mf)
    in2=np.where(f>mf)
    n00,n01=nos(c,in1)
    n10,n11=nos(c,in2)
    tots=n00+n01+n10+n11
    if n00+n01==0:
        val1=0
    else:
        val1=val(n00,n01,tots)
        
    if n10+n11==0:
        val2=0
    else:
        val2=val(n10,n11,tots)
    spinf=val1+val2
    return spinf

In [ ]:
splitinfoval=np.zeros((56,20))
fileloc='/'
for i in range(1,57):
    print(i)
    fname=str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',')
    for j in range(0,20):
        splitinfoval[i-1,j]=splitinfo(data[:,j],data[:,-1])
fname=fileloc+'splitinfo.csv'
np.savetxt(fname,splitinfoval, delimiter=',', fmt='%f')

In [ ]:
rgainratioval=np.zeros((56,25))
fileloc='/'
for i in range(1,57):
    print(i)
    fname1=fileloc + 'splitinfo.csv'
    fname2=fileloc + 'infogain.csv'
    data2=np.genfromtxt(fname1,delimiter=',')
    data1=np.genfromtxt(fname2,delimiter=',')
    dicts = {}
    keys = range(20)
    values = {}
    for j in range(0,20):
        if data2[i-1,j]==0:
            rgainratioval[i-1,j]=0
            values[j]=rgainratioval[i-1,j]
        else:
            rgainratioval[i-1,j]=data1[i-1,j]/data2[i-1,j]
            values[j]=rgainratioval[i-1,j]
            
    for k in keys:
        dicts[k] = values[k]
        
    ans = sorted(dicts.items(),reverse=True,key=lambda dicts:dicts[1])[0:5]
    rgainratioval[i-1,20]=ans[0][0]+1
    rgainratioval[i-1,21]=ans[1][0]+1
    rgainratioval[i-1,22]=ans[2][0]+1
    rgainratioval[i-1,23]=ans[3][0]+1
    rgainratioval[i-1,24]=ans[4][0]+1
fname=fileloc+'rankedgainratio.csv'
np.savetxt(fname,rgainratioval, delimiter=',', fmt='%f')